In [ ]:
%pip install transformers

In [ ]:
import os
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

directory = '/notebooks/xxx'

In [ ]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



In [ ]:

for filename in os.listdir(directory):
    # Check if the file is an image (you may need to add more file types depending on your image formats)
    if filename.endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        # Get the name of the image without the extension
        name_without_extension = os.path.splitext(filename)[0]
        # Create a text file with the same name
        with open(f'{directory}/{name_without_extension}.txt', 'w') as txt_file:
            txt_file.write(name_without_extension)

In [ ]:
max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

def predict_step(directory_path):
    image_paths = [f'{directory_path}/{filename}' for filename in os.listdir(directory_path) if filename.endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
    images = []
    for image_path in image_paths:
        i_image = Image.open(image_path)
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")
        images.append(i_image)

    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    
    for image_path, pred in zip(image_paths, preds):
        name_without_extension = os.path.splitext(os.path.basename(image_path))[0]
        with open(f'{directory_path}/{name_without_extension}.txt', 'w') as txt_file:
            txt_file.write(pred)

# Call the function with the directory path
predict_step(directory)

In [ ]:
def prepend_text(directory_path, text_to_prepend):
    for filename in os.listdir(directory_path):
        # Check if the file is a text file
        if filename.endswith('.txt'):
            file_path = f'{directory_path}/{filename}'
            # Read the existing content of the file
            with open(file_path, 'r') as txt_file:
                content = txt_file.read()
            # Prepend the desired text
            updated_content = f'{text_to_prepend} {content}'
            # Rewrite the file with the updated content
            with open(file_path, 'w') as txt_file:
                txt_file.write(updated_content)

# Specify the text to prepend
text_to_prepend = 'ppxxxtarus'

# Call the function with the directory path and the text to prepend
prepend_text(directory, text_to_prepend)